# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
#probability is P/1-P (probability of something happening over probability of something not happening
#can divide one calculation by another to get an overall probability
#result would be in the order of -1 through to 1)
#google simpsons paradox for more info on the above. 
#
#logistic regression -> log(P/1-P)= B0,B1,X0,X1+..+E
# eg, log(Pdeath/1-Pdeath)=Bo,B1,Xlitres of sugar, Xsmoker+E

#for example, use spam dataset

In [35]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
import statsmodels.formula.api as smf
import scipy.stats as stats
from patsy import dmatrices




In [36]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw




In [37]:

df.head()


,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [38]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df['admit'], df['prestige'], rownames=['admit']).apply(lambda x: x/x.sum(), axis=0)

#df.describe()

#.apply(lambda )


prestige,1.0,2.0,3.0,4.0
admit,,,,
0,0.459016,0.646667,0.768595,0.820896
1,0.540984,0.353333,0.231405,0.179104


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [39]:
dummy_ranks = (pd.get_dummies(df['prestige'], prefix='prestige'))
dummy_ranks.head()



,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


In [40]:
new_cols = ['admit', 'gre', 'gpa']
data = df[new_cols].join(dummy_ranks.ix[:, 'prestige_1':])
data.head()


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: #Two, admit = 0 or admit = 1

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [41]:
cols_for_odds = ['admit', 'gre', 'gpa']
handCalc = df[cols_for_odds].join(dummy_ranks.ix[:, 'prestige_1':])
handCalc.head()



,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


In [42]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [43]:
cross_tab=pd.crosstab(df['admit'], df['prestige'], rownames=['admit'])
cross_tab

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,97,93,55
1,33,53,28,12


In [44]:
cross_tab.iloc[0,2]

93

In [45]:
odds_admit1 =cross_tab.iloc[0,0]
print (odds_admit1)

28


In [46]:
odds_noadmit1 = cross_tab.iloc[1,0]
print (odds_noadmit1)

33


In [47]:
admit_odds1 = odds_admit1/(odds_noadmit1+odds_admit1)
print (admit_odds1)

0.45901639344262296


Odds of being admitted to grad school if you attended a #1 college are 46%

In [48]:


aggr_df = df.groupby('prestige')['admit'].agg(['count'])
aggr_df.head()


,count
prestige,
1.0,61
2.0,150
3.0,121
4.0,67


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [49]:
cross_tab=pd.crosstab(df['admit'], df['prestige'], rownames=['admit'])
cross_tab

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,97,93,55
1,33,53,28,12


In [50]:
odds_not1admit = np.sum(cross_tab.iloc[0,1:])
print (odds_not1admit)

245


In [80]:
odds_not1noadmit = np.sum(cross_tab.iloc[1,1:])
print (odds_not1noadmit)

93


In [52]:
not1odds = odds_not1noadmit/odds_not1admit
print(not1odds)

0.3795918367346939


Odds of admission if someone did not attend a #1 ranked college is 38%

#### 3.3 Calculate the odds ratio

In [82]:
from sklearn.linear_model import LogisticRegression
lm = LogisticRegression()

In [83]:
y = df['admit'].values
X = df['prestige'].values

print (not1odds)



0.3795918367346939


In [85]:
#not sure how to do this

#### 3.4 Write this finding in a sentenance: 

Answer: The odds of being admitted are .316:1

#### 3.5 Print the cross tab for prestige_4

In [86]:
print (pd.crosstab(df['admit'], 'cross_tab.iloc[,3]', rownames=['admit']))

col_0  cross_tab.iloc[,3]
admit                    
0                     273
1                     127


#### 3.6 Calculate the OR 

In [ ]:
#per 3.3, not sure how to do this

#### 3.7 Write this finding in a sentence

Answer: Odds of being addmitted to prestige 4 is .179:1

## Part 4. Analysis

In [88]:
# create a clean data frame for the regression
cols = ['admit', 'gre', 'gpa']
data = df[cols].join(dummy_ranks.ix[:, 'prestige_1':])
print (data.head())

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [93]:
# manually add the intercept
data['intercept'] = 1

#### 4.1 Set the covariates to a variable called train_cols

In [97]:
#not sure how to do this

/anaconda3/lib/python3.6/site-packages/statsmodels/base/data.py:131: RuntimeWarning: invalid value encountered in reduce
  const_idx = np.where(self.exog.ptp(axis=0) == 0)[0].squeeze()
/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:26: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims)
/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1647: RuntimeWarning: invalid value encountered in greater
  return count_nonzero(S > tol, axis=-1)


#### 4.2 Fit the model

In [92]:
#not sure how to do this

/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:1647: RuntimeWarning: invalid value encountered in greater
  return count_nonzero(S > tol, axis=-1)


#### 4.3 Print the summary results

In [ ]:
#per above, not sure how to do this

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [ ]:
#not sure how to do this

#### 4.5 Interpret the OR of Prestige_2

Answer: per earlier responses, unable to do this

#### 4.6 Interpret the OR of GPA

Answer: per earlier responses, unable to do this

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [98]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [99]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(gres)? (<ipython-input-99-5012bd9be7f5>, line 4)

#### 5.1 Recreate the dummy variables

In [ ]:
# recreate the dummy variables

# keep only what we need for making predictions


#### 5.2 Make predictions on the enumerated dataset

#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.

In [ ]:

#unable to do any of 5, nor this one